# Mass Segregation for NGC 7492

We will try tos tudy the mass segregation effect and spatial distribution of stars around the Galactic globular cluster NGC 7492 using the ??? photometry data. NGC 7492 is one of the most sparse globular clusters and is located far from the Galactic centre and plane ($R_{GC} = 24.9 kpc$, $Z = -23.1 kpc$) at 26.2 kpc from the Sun.

| Parameter                             | Value      |  Reference |
|---------------------------------------|------------|------------|
| $\alpha$(J2000)...................... | 23 08 26.7 | 1          |
| $\delta$(J2000)...................... | -15 36 41  | 1          |
| $l$(J2000)(deg)..............         | 53.39      | 1          |
| $b$(J2000)(deg)..............         | -63.48     | 1          |
| $log \rho_0 (M_\odot\ pc^{-3})$... .. | 0.97       | 1          |
| $r_t$(arcmin).....................    | 8.35       | 1          |
| c = $log(r_t/r_c)$.............       | 1.00       | 1          |
| [Fe/H]...........................     | -1.51      | 2          |
| (m - M)_0......................       | 17.09      | 2          |
| E(B - V)........................      | 0.00       | 2          |

## Import dependencies
* pandas
* bokeh
* scipy

In [ ]:
if True:
    import pandas as pd
    from bokeh.layouts import row, column
    from bokeh.models import Select, Label, ColumnDataSource, HoverTool, Whisker
    from bokeh.palettes import Spectral5
    from bokeh.plotting import curdoc, figure, show, output_file
    from bokeh.io import output_notebook
    from bokeh.sampledata.autompg import autompg_clean as df
    import scipy as sp
    from tabulate import tabulate

## 1 Data handling

Data was already handled (see other ntbk). Contains some extra columns, described next:

* Radius_cmd: A radius based on luminosity, for the CMD plot.
* g-r: Difference in g and r magnitudes. It's the color.
* Color: rgb value for plotting color in the CMD
* RA_arcmin: RA in arcmin, centered.
* DEC_arcmin: DEC in arcmin, centered.
* Distance_c: Aproximate distance from cluster center. Deprecated by rd. Glad to see they are not so different.
* grid_x: Grid x coordinate
* grid_y: Grid y coordinate
* vip: True if in the final grid selection

In [ ]:
df = pd.read_pickle('dataframe_v2')

#df = sp.loadtxt('ngc7492_calib_cut.erad')
#head = ['ID', 'RA', 'DEC', 'g', 'gerr', 'r', 'rerr', 'chi', 'sharp',
#        'rd','l', 'b', 'egr', 'ag', 'ar']
#df = pd.DataFrame(df, columns=head)

x = (df['g']-df['r']).values
y = df['r'].values

radii = (y-min(y)+0.00001) / (max(y)-min(y)) * 0.02

rg = (x-min(x))/(max(x)-min(x)) * 250
#rg *= sp.fabs( sp.sin((rg-125)/125 * sp.pi/2 ))
colors = ["#%02x%02x%02x" % (int(r), int(g), 0) for r, g in zip(rg, 250-rg)]

#df.insert(len(df.T), 'g-r', x)
#df.insert(len(df.T), 'Radius_cmd', radii)
#df.insert(len(df.T), 'Color', colors)

## 2 Define Variables, Functions and Masks
Just that, definitions and some variables for the study

### Plot Options

In [ ]:
plot_scale = 7
TOOLS="pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,save,hover,"

# Plots to draw
circles = True
CMD0 = True
CMD_inner = True
CMD_outer = True

### Variables
Some of them are explained later, but are put here for ease of use.

In [ ]:
N = len(df)
dm = 17.09
FeH = -1.51
MH = FeH + 0.35


box_y_min = 15
box_y_max = 25.5
box_x_min = -0.5
box_x_max = 1.1

grid_nx = 32
grid_ny = 50
grid_dx = 0.05
grid_dy = 0.2

dx = sp.arange(grid_nx+1) * grid_dx + box_x_min
dy = sp.arange(grid_ny+1) * grid_dy + box_y_min

inner_radius = 8.35
outer_radius = 8.35 * 2.5

# build center and tidal radius
ra_am = df['RA'] / 24 * 360 * 60 # hh.hhhhhh to arcmin    
dec_am = df['DEC'] * 60  # dd.dddddd to arcmin

ra_mean = sp.mean(ra_am)
dec_mean = sp.mean(dec_am)

ra_f = ra_am - ra_mean
dec_f = dec_am - dec_mean

#df.insert(len(df.T), 'RA_arcmin', ra_f)  # ra in arcmin mean substracted
#df.insert(len(df.T), 'DEC_arcmin', dec_f)  # dec in arcmin mean substracted

ra0 = 347.11117 * 60  # center
dec0 = -15.61147 * 60  # center

ra_0 = ra0 - ra_mean  # center mean substracted
dec_0 = dec0 - dec_mean  # center mean substracted


mask_max_mag = df['r'] <= 26
mask_inner = df['rd'] < inner_radius
mask_outer = df['rd'] > outer_radius

mask_mag_min = box_y_min < df['r']
mask_mag_max = box_y_max > df['r']
mask_color_min = box_x_min < df['g-r']
mask_color_max = box_x_max > df['g-r']

mask_box = mask_mag_min&mask_mag_max&mask_color_min&mask_color_max

### Plot Functions

In [ ]:
def plot_cdm(A, plot_scale=plot_scale, TOOLS=TOOLS, add_box=False, add_grid=None,
             title = 'CMD Complete Sample', axis=sp.array([]), accommodate=False):

    source1 = ColumnDataSource(A)

    # figure and tools

    p = figure(tools=TOOLS, title=title, toolbar_location='below')
    p.plot_height = 100 * plot_scale
    p.plot_width = 100 * plot_scale
    p.y_range.flipped = True

    hover = p.select(dict(type=HoverTool))
    hover.tooltips = [
        ('ID', '@ID'),
        ('RA(2000.0), DEC(2000.0)', '@RA, @DEC}'),
        ('g-r', '@{g-r}'),
        ('r', '@r'),
        ('chi', '@chi'),
        ('sharp', '@sharp')
        ]

    # title
    p.title.text_font_size = '24pt'
    p.title.align = "center"

    #axis
    p.xaxis.axis_label = 'g - r'
    p.yaxis.axis_label = 'r'
    p.xaxis.axis_label_text_font_size = "22pt"
    p.yaxis.axis_label_text_font_size = "22pt"

    p.xaxis.major_label_text_font_size = "20pt"
    p.yaxis.major_label_text_font_size = "20pt"

    p.scatter('g-r', 'r', source=source1, radius='Radius_cmd', fill_color='Color',
              fill_alpha=0.8, line_color=None, legend='N = %i' % len(A))
    if accommodate:
        p.scatter([-1, 2], [20, 20], fill_color='white', fill_alpha=0., line_color=None)
    if add_box:
        p.segment(x0=[box_x_min, box_x_max, box_x_max, box_x_min], y0=[box_y_min, box_y_min, box_y_max,box_y_max], x1=[box_x_max, box_x_max, box_x_min, box_x_min],
          y1=[box_y_min, box_y_max,box_y_max,box_y_min], color="gray", line_width=2, alpha=0.8)
        pass
    if add_grid is not None:
        for x in add_grid:
            p.quad(top=dy[x[0]+1], bottom=dy[x[0]], left=dx[x[1]], right=dx[x[1]+1], color="#B3DE69", alpha=0.3)

    p.legend.location = "top_right"
    p.legend.click_policy="hide"
    p.legend.label_text_font_size = '18pt'
    # save and display
    output_notebook()
    #output_file("colormag_diag.html", title="Color Magnitude Diagram")
    show(p)  # open a browser
    
def circles(df0):
    N24 = len(df0)
    
    source2 = ColumnDataSource(df0)

    p1 = figure(tools=TOOLS, title="Observed Fields r < 26", toolbar_location='below')

    p1.plot_height = 100 * 7
    p1.plot_width = 100 * 7
    p1.x_range.flipped = True
    bokeh_text = Label(x=-9.5, y=29.9, text='N = %i' % N24, text_font_size='20pt')
    p1.add_layout(bokeh_text)

    bokeh_text = Label(x=-4, y=sp.sqrt(56), text="r=8'.35", text_font_size="16pt", text_color="black",
                       text_font_style='bold')
    bokeh_text1 = Label(x=-10, y=sp.sqrt(335), text="r=20'.875", text_font_size="16pt", text_color="black",
                        text_font_style="bold")

    p1.add_layout(bokeh_text)
    p1.add_layout(bokeh_text1)

    # title
    p1.title.text_font_size = '24pt'
    p1.title.align = "center"

    #axis
    p1.xaxis.axis_label = 'RA (arcmin)'
    p1.yaxis.axis_label = 'DEC (arcmin)'
    p1.xaxis.axis_label_text_font_size = "20pt"
    p1.yaxis.axis_label_text_font_size = "20pt"

    p1.xaxis.major_label_text_font_size = str(int(16*7/plot_scale))+"pt"
    p1.yaxis.major_label_text_font_size = "18pt"

    p1.scatter('RA_arcmin', 'DEC_arcmin', source=source2, size='Radius_circles', alpha=0.6)
    p1.scatter(ra_0, dec_0, marker='o+', line_color="red", fill_color="red", size=3)
    
    p1.arc(ra_0, dec_0, radius=inner_radius, start_angle=0, end_angle=2*sp.pi, color='red')
    p1.arc(ra_0, dec_0, radius=outer_radius, start_angle=0, end_angle=2*sp.pi, color='red')
    
    output_notebook()
    #output_file("observed_field.html", title="Observed Field")
    show(p1)  # open a browser

## 3 CMD and simple filterings
The first filtering is taking just the stars brighter than 26 magnitudes in r. This reduces the sample in a 40% aproximately, down to $N =44566$

In [ ]:
plot_cdm(df)

In [ ]:
df0 = df[mask_max_mag]

x = df0['g-r']  # color correction rutine
rg = (x-min(x))/(max(x)-min(x)) * 250
df0.Color = ["#%02x%02x%02x" % (int(r), int(g), 0) for r, g in zip(rg, 250-rg)]

plot_cdm(df0, plot_scale=7, title='CMD r < 26')

## 4 Observed Fields
To get an idea of the cluster we are observing and it's inner and outer radius.

In [ ]:
#radii = (df['r'].values + 0.0001 - min(df['r'].values)) / (max(df['r'].values) - min(df['r'].values))
#radii = radii * 3
#df.insert(len(df.T), 'Radius_circles', radii)

circles(df[mask_max_mag])

## 5 Filters

### Box Filtering
We will take stars in the region $-0.5 < g-r < 1.1$ and $15 < r < 25.5$ and make a grid of (color, mag) of dimentions ($32 x 50$), with a spacing of $(0.05, 0.2)$, eliminating stars too far away from the MS (and not part of the cluster).

In [ ]:
df_box = df[mask_max_mag&mask_box]

x = df_box['g-r']
rg = ( (x-min(x)) / (max(x)-min(x)) ) * 250
df_box.Color = ["#%02x%02x%02x" % (int(r), int(g), 0) for r, g in zip(rg, 250-rg)]

plot_cdm(df_box, plot_scale = 7, title='CMD of boxed area',add_box=True, accommodate=True)

### Inner and outer regions
That way, we have $N_{cl}=8104$ stars in the inner region ($r<r_t=8.'35$) and $N_{f}=20236$ stars in the outer region ($r>2.5r_t=20.'9$)

In [ ]:
df1 = df[mask_max_mag&mask_box&mask_inner]
df2 = df[mask_max_mag&mask_box&mask_outer]

x1 = df1['g-r']
rg1 = ( (x1-min(x1)) / (max(x1)-min(x1)) ) * 250
df1.Color = ["#%02x%02x%02x" % (int(r1), int(g1), 0) for r1, g1 in zip(rg1, 250-rg1)]

x2 = df2['g-r']
rg2 = ( (x2-min(x2)) / (max(x2)-min(x2)) ) * 250
df2.Color = ["#%02x%02x%02x" % (int(r2), int(g2), 0) for r2, g2 in zip(rg2, 250-rg2)]


plot_cdm(df1, plot_scale=6, title='CMD inner region', accommodate=True)
plot_cdm(df2, plot_scale=6, title='CMD outer region', accommodate=True)

### Grid specifications
As said, is a 32x50 matrix. We will apply in this space the following filtering to select just the stars in some squares of the grid:

#### Color Magnitude Sequence
Assuming that color-mag distribution of field stars is constant across the observed field, a color-mag sequence is estimated (Grillmair et al. 1995):
$$f_{cl}(i, j) = n_{cl}(i,j) - gn_f(i,j)$$
Where $n$ is number of stars, subindex $cl$ represents the cluster and $f$ the field stars, g is the ratio of the area of the cluster to the background field region.
Although the contribution of foreground and background stars in our frame is expected to be small as a result of the cluster’s large distance from the galactic plane.

#### S/N
S/N of the expected true number of stars per subarea of the grid:
$$S/N(i,j) = \frac{f_{cl}(i,j)}{\sqrt{(n_{cl}(i,j) + g^2n_f(i,j))}}$$

We have the squares and compute individual SN for each, we sort them out in descending order, and calculate cummulative SN in that array.
We compute the maximum in that SN cummulative series, and take only those grid squares. This squares are signales as the 'vip'.

In [ ]:
'''
# Insert grid columns


df.insert(len(df.T), 'grid_x', sp.ones(len(df))*-sp.inf)
df.insert(len(df.T), 'grid_y', sp.ones(len(df))*-sp.inf)
df.insert(len(df.T), 'vip', False)
for x in range(grid_nx):
    for y in range(grid_ny):
        selected = (dx[x] <= df['g-r'])&(df['g-r']<= dx[x+1])&(dy[y] <= df['r'])&(df['r']<= dy[y+1])
        df['grid_x'][selected] = int(y)
        df['grid_y'][selected] = int(x)
        #print('\r'+str(y/50*100)+'%')
    print(str(x/32*100)+'%')
'''

In [ ]:
n_cl, n_f = sp.zeros((grid_ny, grid_nx)), sp.zeros((grid_ny, grid_nx))

for x in range(grid_nx):
    for y in range(grid_ny):
        n_cl[y, x] = sp.sum((df1.grid_x == y)&(df1.grid_y == x))
        n_f[y, x] = sp.sum((df2.grid_x == y)&(df2.grid_y == x))

area_cl = sp.pi * inner_radius ** 2
area_f = sp.pi * (max(df2['rd']) ** 2 - outer_radius ** 2)
g = area_cl / area_f

f = n_cl - g * n_f
SN = f / (sp.sqrt(n_cl + g**2*n_f))

SNl = SN.reshape(-1)
SNl = sp.where(sp.isnan(SNl)==False, SNl, -sp.inf)
order = sp.argsort(SNl)[::-1]
SNl = SNl[order]
n_clk = n_cl.reshape(-1)[order]
n_fk = n_f.reshape(-1)[order]

SNk = sp.zeros_like(SNl)
n_fk

for i in range(len(SNk)):
    a = (sp.sum(n_clk[:i]) - g * sp.sum(n_fk[:i])) / (sp.sqrt(sp.sum(n_clk[:i]) + g**2 * sp.sum(n_fk[:i])))
    if sp.isnan(a):
        SNk[i] = 0
    else:
        SNk[i] = a
d = sp.argsort(SNk)[::-1]
d_max = d[0]
## NOTE: 'RuntimeWarning: invalid value encountered in true_divide' nan results are considered!

In [ ]:
d_max

In [ ]:
vip_set = order[:d_max]

vip_pairs = sp.array([[v//grid_nx, v%grid_nx] for v in vip_set])

# This is done in the supplementary jupyter, for speed it has been commented as its appended as the vip column.
'''
count = 0
for cherry in vip_pairs:
    for DF in [df]:
        DF.vip[(DF['grid_x']==cherry[0])&(DF['grid_y']==cherry[1])] = True
    count += 1
    print(str(int(count/len(vip_pairs)*100))+'%')

df.to_pickle('dataframe_v2')
'''

### Grid data
Now we use only the stars in the grid, and we are down to $N_{cl}=7933$ and $N_f=16397$. Several field stars have been left out with certainty and few of the inner region (~4000 and ~700 respectively!).

In [ ]:
df1v = df1[df1.vip==True]
df2v = df2[df2.vip==True]

plot_cdm(df1v, plot_scale=6, title='CMD inner region', accommodate=True, add_grid=vip_pairs)
plot_cdm(df2v, plot_scale=6, title='CMD outer region', accommodate=True, add_grid=vip_pairs)

## 6 Mass Segregation
Radial variation of of LFs and MFs.
To investigate any spatial variation in the LF, we choose a critical radius and separate in inner and outer region of the cluster, such that they have a similar number of stars.

We will construct two separate LFs, $LF_{inner} = r<1'.3$ and $LF_{outter} = 1'.3 < r < 8'.35$

**Where r_c = 1.6, despite almost everywhere saying it's 1.3**

In [ ]:
r_crit = 1.6

mask_lf_in = df['rd'] <= r_crit
mask_lf_out = (df['rd'] > r_crit)&(inner_radius > df['rd'])
mask_mag20 = df['r'] > 20
mask_vip = df.vip==True

#df = df[mask_mag_max&mask_box&mask_inner]

df_all = df1v#[mask_mag20]
df_inner = df[mask_mag_max&mask_box&mask_inner&mask_vip&mask_lf_in]#&mask_mag20]
df_outer = df[mask_mag_max&mask_box&mask_inner&mask_vip&mask_lf_out]#&mask_mag20]

sp.sum(df_inner.vip), sp.sum(df_outer.vip)

### Incompleteness Corrections

* Incompleteness factors per color are independent or dependent?
The factor is the ratio of recovered stars with injected stars, per magnitude bin.


Correction factors are included empirically according to Lee2004 as follows:

| ......... 	| Inner 0 < r <1.3 	| .... 	| Outer (1.3 < r < 8.35) 	| .... 	| Global 	| .... 	|
|-----------	|------------------	|------	|------------------------	|------	|--------	|------	|
| g         	| N                	| f    	| N                      	| f    	| N      	| f    	|
| 15.5-16.5 	| 4.0              	| 1.0  	| 3.0                    	| 1.0  	| 7.0    	| 1.0  	|
| 16.5-17.5 	| 10.0             	| 1.0  	| 12.0                   	| 1.0  	| 22.0   	| 1.0  	|
| 17.5-18.5 	| 32.0             	| 1.0  	| 26.0                   	| 1.0  	| 58.0   	| 1.0  	|
| 18.5-19.5 	| 28.0             	| 1.0  	| 22.0                   	| 1.0  	| 50.0   	| 1.0  	|
| 19.5-20.5 	| 73.2             	| 0.99 	| 49.5                   	| 0.99 	| 122.7  	| 0.98 	|
| 20.5-21.5 	| 490.0            	| 0.90 	| 310.4                  	| 0.96 	| 800.4  	| 0.94 	|
| 21.5-22.5 	| 963.3            	| 0.79 	| 669.2                  	| 0.91 	| 1632.5 	| 0.87 	|
| 22.5-23.5 	| 1263.8           	| 0.69 	| 943.3                  	| 0.90 	| 2207.1 	| 0.80 	|
| 23.5-24.5 	| 1274.4           	| 0.43 	| 1068.6                 	| 0.70 	| 2343.0 	| 0.58 	|

In [ ]:
x0, y0 = sp.histogram(df_all['r'], sp.linspace(15, 24, 10))  # total
x1, y1 = sp.histogram(df_inner['r'], sp.linspace(15, 24, 10))  # inner
x2, y2 = sp.histogram(df_outer['r'], sp.linspace(15, 24, 10))  # outer

correction = pd.DataFrame(data={'r':['15.5-16.5', '16.5-17.5', '17.5-18.5', '18.5-19.5', '19.5-20.5', '20.5-21.5', '21.5-22.5', '22.5-23.5', '23.5-24.5'],
                                'N':[7, 22, 58, 50, 123, 800, 1633, 2207, 2343],
                                'f':[1, 1, 1, 1, 0.98, .94, 0.87, .80, .58],
                                'N_i':[4, 10, 32, 28, 73, 490, 963, 1264, 1274],
                                'f_i':[1, 1, 1, 1, .99, .9, .79, .69, .43],
                                'N_o':[3, 12, 26, 22, 50, 310, 669, 943, 1069],
                                'f_o':[1, 1, 1, 1, .99, .96, .91, .9, .7]})

x0c = x0# + correction.N.values
x1c = x1# + correction.N_i.values
x2c = x2# + correction.N_o.values
x2c

### Luminosity Function

Points are the counts per bin per region, steps are the total for both regions, dashed line is the results in CRF91, but in fking V, displaced here by a constant.

In [ ]:
lf_CRF = pd.DataFrame(data={'v':sp.linspace(15.5, 23.5, 9),
                            'n':[1.9, 8.9, 37.4, 44.1, 46.7, 338.7, 1032.6, 1403.4, 1729.6]})

In [ ]:
plot_scale = 4
if True:
    p1 = figure(tools=TOOLS, title="Luminosity Function 0<r<%.1f" % r_crit, toolbar_location='below', y_axis_type="log")
    p1.plot_height = 100 * plot_scale
    p1.plot_width = 100 * plot_scale

    # title
    p1.title.text_font_size = '13.5pt'
    p1.title.align = "center"

    #axis
    p1.xaxis.axis_label = 'r'
    p1.yaxis.axis_label = 'log N + const.'
    p1.xaxis.axis_label_text_font_size = "15pt"
    p1.yaxis.axis_label_text_font_size = "15pt"

    p1.xaxis.major_label_text_font_size = "13pt"
    p1.yaxis.major_label_text_font_size = "13pt"

    p1.scatter(sp.linspace(16, 24, 9), x1c, color='black')
    p1.step(sp.linspace(16, 24, 9), x0c * sp.sum(x1c) / sp.sum(x0c), line_width=2, mode="center", color='gray')
    
    p1.line(lf_CRF.v, lf_CRF.n+4, line_alpha=0.7, line_color='orange', line_dash='dashed')
    
    #err_xs, err_ys = [], []
    #for x, y, yerr in zip(sp.linspace(16, 24, 9), x1c, x1c*0.25):
    #    err_xs.append((x, x))
    #    err_ys.append((y - yerr, y + yerr))

    # plot them
    #p1.multi_line(err_xs, err_ys, color='black')
    
    #p1.scatter(ra_0, dec_0, marker='o+', line_color="red", fill_color="red", size=3)
    p2 = figure(tools=TOOLS, title="Luminosity Function %.1f<r<8.35" % r_crit, toolbar_location='below', y_axis_type="log")
    p2.plot_height = 100 * plot_scale
    p2.plot_width = 100 * plot_scale

    # title
    p2.title.text_font_size = '13.5pt'
    p2.title.align = "center"

    #axis
    p2.xaxis.axis_label = 'r'
    p2.yaxis.axis_label = 'log N + const.'
    p2.xaxis.axis_label_text_font_size = "15pt"
    p2.yaxis.axis_label_text_font_size = "15pt"

    p2.xaxis.major_label_text_font_size = "13pt"
    p2.yaxis.major_label_text_font_size = "13pt"

    p2.scatter(sp.linspace(16, 24, 9), x2c, color='black')
    p2.step(sp.linspace(16, 24, 9), x0c * sp.sum(x2c) / sp.sum(x0c), line_width=2, mode="center", color='gray')
    p2.line(lf_CRF.v, lf_CRF.n+6, line_alpha=0.7, line_color='orange', line_dash='dashed')
    
    output_notebook()
    #output_file("observed_field.html", title="Observed Field")
    show(row(p1, p2))  # open a browser

## Mass Function
Baraffe et al 1997, we take only r > 20 stars, since the counts are so low for brigther stars

In [ ]:
df_m0 = df[mask_mag_max&mask_box&mask_inner&mask_vip&mask_inner&mask_mag20]
df_m1 = df[mask_mag_max&mask_box&mask_inner&mask_vip&mask_lf_in&mask_mag20]
df_m2 = df[mask_mag_max&mask_box&mask_inner&mask_vip&mask_lf_out&mask_mag20]

# reroll colors
aux1 = df_m1['g-r']
rg1 = ( (aux1-min(aux1)) / (max(aux1)-min(aux1)) ) * 250
df_m1.Color = ["#%02x%02x%02x" % (int(r1), int(g1), 0) for r1, g1 in zip(rg1, 250-rg1)]

aux2 = df_m2['g-r']
rg2 = ( (aux2-min(aux2)) / (max(aux2)-min(aux2)) ) * 250
df_m2.Color = ["#%02x%02x%02x" % (int(r2), int(g2), 0) for r2, g2 in zip(rg2, 250-rg2)]


plot_cdm(df_m1, plot_scale=6, title='CMD inner region', accommodate=True)
plot_cdm(df_m2, plot_scale=6, title='CMD outer region', accommodate=True)

In [ ]:
v = sp.linspace(16, 25, 10)
x0, y0 = sp.histogram(df_m0['r'], v)
x1, y1 = sp.histogram(df_m1['r'], v)
x2, y2 = sp.histogram(df_m2['r'], v)

x0c = x0 * correction.f.values# + correction.N.values
x1c = x1 * correction.f_i.values# + correction.N_i.values
x2c = x2 * correction.f_o.values# + correction.N_o.values

sp.log10(x1c[-5:])/-1.1
M = v - dm
#(sp.log10(M)/-1.1)[-5:]
#10**sp.array([-0.1, -0.2, -0.3])
#vv = sp.array([20,21,22,23,24,25])-17
x1[-5:], y1[-5:]-17

vv = sp.array([4, 4.72, 5.27, 6.27, 7.44, 8.42])
mm = sp.log10(sp.array([0.8, .75, .7, .6, .5]))
X, Y = sp.histogram(df_m0['r']-dm, vv)
X1, Y1 = sp.histogram(df_m1['r']-dm, vv)
X2, Y2 = sp.histogram(df_m2['r']-dm, vv)

mm

In [ ]:
a = -1.1
m1 = sp.log10(X1)
m2 = sp.log10(X2)

plot_scale = 5
if True:
    p1 = figure(tools=TOOLS, title="Luminosity Function 0<r<%.1f" % r_crit, toolbar_location='below')#, y_axis_type="log")
    p1.plot_height = 100 * plot_scale
    p1.plot_width = 100 * plot_scale

    # title
    p1.title.text_font_size = '13.5pt'
    p1.title.align = "center"
    
    #axis
    p1.x_range.flipped = True

    p1.xaxis.axis_label = 'log m'
    p1.yaxis.axis_label = 'log N + const.'
    p1.xaxis.axis_label_text_font_size = "15pt"
    p1.yaxis.axis_label_text_font_size = "15pt"

    p1.xaxis.major_label_text_font_size = "13pt"
    p1.yaxis.major_label_text_font_size = "13pt"

    p1.scatter(mm, sp.log10(X1), color='black')    
    p1.circle(mm, sp.log10(X2), color='black', fill_color='White')
    
    output_notebook()
    #output_file("observed_field.html", title="Observed Field")
    show(p1)  # open a browser